# Using scaled values

Sometimes, scales can just be given to a widget, and it will be used internally by that widget to scale its values as needed. However, to integrate with other widgets that do not know about scales, we use the `ScaledValue` widget as a link.

In [1]:
from ipyscales import LinearScale, QuantizeScale, NamedOrdinalColorMap, ScaledValue
from ipywidgets import FloatSlider, FloatRangeSlider, FloatText, Label, ColorPicker, jslink, GridBox, HBox, Layout
from traitlets import link

## Scaling a float input

For our first example, we start with a simple linear normalizer scale:

In [2]:
normalizer = LinearScale(domain=(0, 100))

We then add a range slider to select its input domain. The scale will then normalize this domain to its default `[0, 1]` range.

In [3]:
domain_picker = FloatRangeSlider(value=normalizer.domain, min=-100, max=100)
jslink((normalizer, 'domain'), (domain_picker, 'value'))
domain_picker

FloatRangeSlider(value=(0.0, 100.0), min=-100.0)

To show the effect of this normalizer on a value, we set up an input value from a slider, and a **scaled version of that value** .

In [4]:
input_value = FloatSlider(value=0, min=domain_picker.min, max=domain_picker.max)
link((input_value, 'min'), (domain_picker, 'min'))
link((input_value, 'max'), (domain_picker, 'max'))

normalized_value = ScaledValue(scale=normalizer)
jslink((input_value, 'value'), (normalized_value, 'input'))

input_value

FloatSlider(value=0.0, min=-100.0)

To display the scaled value, we link it to a display. We disable the input of the display, as it does not make sense for the user to write to this value direcly.

In [5]:
value_display = FloatText(disabled=True)
jslink((normalized_value, 'output'), (value_display, 'value'))
value_display

FloatText(value=0.0, disabled=True)

## Other scales

We can also use the `ScaledValue` widget with other kinds of scales. Here, we set up a quantize scale to split the domain above into four equally sized regions, that each map to a string (a label for a quartile):

In [6]:
quartile = QuantizeScale(range=('first', 'second', 'third', 'fourth'))
jslink((normalizer, 'domain'), (quartile, 'domain'))

Next, we set up a `ScaledValue` for mapping our user input through this scale:

In [7]:
quartile_value = ScaledValue(scale=quartile)
jslink((input_value, 'value'), (quartile_value, 'input'))

And finally, we link the scaled output to a `Label` to display it to the user. Note how any values outside the domain will be clamped to the nearest edge value.

In [8]:
quartile_display = Label()
jslink((quartile_value, 'output'), (quartile_display, 'value'))
quartile_display

Label(value='')

## Chaining scaled values

Sometimes, it might make sense to chain multiple scaled values. Here, we will continue the quartile example above by setting up an ordinal color scale which maps the quartile label above to a color. While it would be possible to set up a scale that maps a value directly from the input range to an ordinal color, it might be easier to simply set up a chain:

In [9]:
color_scale = NamedOrdinalColorMap('Spectral', cardinality=4)
jslink((quartile, 'range'), (color_scale, 'domain'))
color_scale.edit()

In [10]:
color_value = ScaledValue(input=quartile_value, scale=color_scale)

In [11]:
colored_display = ColorPicker(disabled=True, concise=True, layout=Layout(width='30px'))
jslink((color_value, 'output'), (colored_display, 'value'))
colored_display

ColorPicker(value='black', concise=True, disabled=True, layout=Layout(width='30px'))

## Dashboard

Collecting all our pieces together in a dashboard (as new views on the same models):

In [12]:
GridBox(
    [
        Label('Domain:'), domain_picker,
        Label('Value:'), input_value,
        Label('Quartile:'), HBox([colored_display, quartile_display])
    ],
    layout=Layout(
        width='50%',
        grid_template_columns='6em auto',
        grid_template_areas='''
            "labelA domain"
            "labelB value "
            "labelC scaled"
        '''
    )
)

GridBox(children=(Label(value='Domain:'), FloatRangeSlider(value=(0.0, 100.0), min=-100.0), Label(value='Value…